# 作業 : (Kaggle)鐵達尼生存預測

In [1]:
# 載入套件與資料
import pandas as pd
import numpy as np

data_path = r'E:\GitHub\all_data'
df_train = pd.read_csv(data_path + r'\titanic_train.csv')
df_test = pd.read_csv(data_path + r'\titanic_test.csv')
df_train.shape

(891, 12)

In [2]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()
df.dtypes

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

In [3]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [4]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [5]:
df.describe(include = 'all')

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,1309.000000,1309,1309,1046.000000,1309.000000,1309.000000,1309,1308.000000,295,1307
unique,NaN,1307,2,NaN,NaN,NaN,929,NaN,186,3
top,NaN,"Kelly, Mr. James",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
freq,NaN,2,843,NaN,NaN,NaN,11,NaN,6,914
mean,2.294882,NaN,NaN,29.881138,0.498854,0.385027,NaN,33.295479,NaN,NaN
std,0.837836,NaN,NaN,14.413493,1.041658,0.865560,NaN,51.758668,NaN,NaN
min,1.000000,NaN,NaN,0.170000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,2.000000,NaN,NaN,21.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,3.000000,NaN,NaN,39.000000,1.000000,0.000000,NaN,31.275000,NaN,NaN


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

## 作業一

In [6]:
# 整數 (int) 特徵取 mean max min
print(f'Mean of int features:\n{df[int_features].mean()}\n')
print(f'Max of int features:\n{df[int_features].max()}\n')
# print(f'Unique of int features:\n{df[int_features].unique()}\n')
#unique() only in series
int_unique = [df[i].unique() for i in int_features]
df_int_unique = pd.Series(list(int_unique), index= int_features,)
print(f'Unique of int features:\n{df_int_unique}\n')

Mean of int features:
Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

Max of int features:
Pclass    3
SibSp     8
Parch     9
dtype: int64

Unique of int features:
Pclass                   [3, 1, 2]
SibSp        [1, 0, 3, 4, 2, 5, 8]
Parch     [0, 1, 2, 5, 3, 4, 6, 9]
dtype: object



In [7]:
# 浮點數 (float) 特徵取 mean max min
print(f'Mean of float features:\n{df[float_features].mean()}\n')
print(f'Max of float features:\n{df[float_features].max()}\n')
# print(f'Max of float features:\n{df[float_features].unique()}\n')
float_unique = [len(df[j].unique()) for j in float_features]
df_float_unique = pd.Series(list(float_unique), index= float_features,)
print(f'number of Unique of float features:\n{df_float_unique}\n')

Mean of float features:
Age     29.881138
Fare    33.295479
dtype: float64

Max of float features:
Age      80.0000
Fare    512.3292
dtype: float64

number of Unique of float features:
Age      99
Fare    282
dtype: int64



In [9]:
# 物件 (Object) 特徵取 mean max min
print(f'Mean of object features:\n{df[object_features].mean()}\n')
print(f'Max of object features:\n{df[object_features].max()}\n')
# print(f'Max of float features:\n{df[float_features].unique()}\n')
object_unique = [len(df[k].unique()) for k in object_features]
df_object_unique = pd.Series(list(object_unique), index= object_features,)
print(f'number of Unique of object features:\n{df_object_unique}\n')

Mean of object features:
Series([], dtype: float64)

Max of object features:
Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

number of Unique of object features:
Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64



## 作業二

### ANS:像是圖片檔，聲音檔，影像檔，其中動態影像辨識作為麻煩，其中數據可轉會為255像素以float方式呈現